One of the applications of convex optimization in complex variables is phase recovery, a problem in signal processing. The objective is to reconstruct the complex phase of a vector when we are only given the magnitude of some linear measurements.

This problem is orginally a nonconvex optimization problem. Using results from [this paper](https://arxiv.org/abs/1206.0102), the problem can be relaxed to a (complex) semidefinite program (complex SDP).

Given a linear operator $A$ and a vector $b$ of measured amplitudes, define the positive semidefinite hermitian matrix $M = \text{diag}(b) (I - A A^*) \text{diag}(b)$. The problem is:

                minimize < U,M >
                subject to 
                diag(U) = 1
                U in :HermitianSemiDefinite
                
Here the variable $U$ must be hermitian ($U \in \mathbb{H}_n $), and we have a solution to the phase recovery problem if $U = u u^*$ has rank one. Otherwise, the leading singular vector of $U$ can be used to approximate the solution.


In [5]:
using Convex

n = 20
p = 5
A = rand(n,p) + im*randn(n,p)
x = randn(p) + im*randn(p)
# constructing b
b = abs(A*x)

M = diagm(b)*(eye(n)-A*ctranspose(A))*diagm(b)
U = ComplexVariable(n,n)
objective = inner_product(U,M)
c1 = diag(U) == 1 
c2 = U in :SDP
p = minimize(objective,c1,c2)
solve!(p)
U.value

(size(coeff),size(var)) = ((40,40),(40,40))
----------------------------------------------------------------------------
	SCS v1.1.8 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012-2015
----------------------------------------------------------------------------
Lin-sys: sparse-direct, nnz in A = 3181
eps = 1.00e-04, alpha = 1.80, max_iters = 20000, normalize = 1, scale = 5.00
Variables n = 801, constraints m = 1641
Cones:	primal zero / dual free vars: 821
	sd vars: 820, sd blks: 1
Setup time: 4.64e-02s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0|      inf       inf      -nan      -inf       inf       inf  5.43e-03 
   100|      inf       inf      -nan      -inf      -inf       inf  1.69e-01 
   200|      inf       inf      -nan      -inf      -inf       inf  2.86e-01

20×20 Array{Complex{Float64},2}:
 1.0-9.03475e-15im         …           0.885476-0.464681im 
    -0.526391-0.850241im              -0.861197-0.508264im 
     0.974472+0.224507im               0.967196-0.254023im 
     0.416762-0.909015im             -0.0533695-0.998573im 
     0.959936-0.280215im                0.71979-0.694188im 
     0.774432-0.632655im   …           0.391758-0.920065im 
      0.55785-0.829942im               0.108305-0.994116im 
     0.986604-0.163136im               0.797807-0.602909im 
     0.699821-0.714318im               0.287745-0.957705im 
     0.870132-0.492818im               0.541478-0.840713im 
     0.998115+0.0613708im  …           0.912325-0.409463im 
     0.929127-0.369759im                 0.6509-0.75916im  
     0.397068+0.917789im               0.778073+0.62817im  
     0.998079-0.0619533im              0.854986-0.518647im 
     0.909299+0.416144im               0.998536-0.0540488im
     0.760578+0.649252im   …           0.975168+0.221471im 
     0.

In [9]:
# Verify if the rank of U is 1
B, C = eig(U.value)
length([b for b in B if(abs(real(b))>1e-4)])
#Ax = diagm(b)*U.value[]
#F = factorize(U.value)
A*x
diagm(b)*C[1,:]

LoadError: MethodError: no method matching *(::Array{Complex{Float64},1}, ::Array{Complex{Float64},1})[0m
Closest candidates are:
  *(::Any, ::Any, [1m[31m::Any[0m, [1m[31m::Any...[0m) at operators.jl:133
  *{T<:Union{Complex{Float32},Complex{Float64},Float32,Float64},S}([1m[31m::Union{Base.ReshapedArray{T<:Union{Complex{Float32},Complex{Float64},Float32,Float64},2,A<:DenseArray,MI<:Tuple{Vararg{Base.MultiplicativeInverses.SignedMultiplicativeInverse{Int32},N}}},DenseArray{T<:Union{Complex{Float32},Complex{Float64},Float32,Float64},2},SubArray{T<:Union{Complex{Float32},Complex{Float64},Float32,Float64},2,A<:Union{Base.ReshapedArray{T,N,A<:DenseArray,MI<:Tuple{Vararg{Base.MultiplicativeInverses.SignedMultiplicativeInverse{Int32},N}}},DenseArray},I<:Tuple{Vararg{Union{Base.AbstractCartesianIndex,Colon,Int32,Range{Int32}},N}},L}}[0m, ::Union{Base.ReshapedArray{S,1,A<:DenseArray,MI<:Tuple{Vararg{Base.MultiplicativeInverses.SignedMultiplicativeInverse{Int32},N}}},DenseArray{S,1},S